In [31]:
import pandas as pd
import numpy as np

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union

import matplotlib.pyplot as plt
import seaborn as sns

#from geoband.API import *
import folium
import json
import geopandas as gpd
from geopandas import gpd

from folium import plugins
import re
from pandas import ExcelFile

data = pd.read_csv("/Users/seungji/Desktop/dataanalysis/project/data/finaldata0617.csv",encoding="cp949")
data = data.loc[:,['미세먼지(㎍/㎥)','lat', 'lon','기온(℃)', '상대습도( %)','버스승차승객수', '버스하차승객수','대여', '반납', 'traffic', 'road', 'subway', 'vent', 'parking','남10대생활인구수', '여10대생활인구수', '남20대생활인구수', '여20대생활인구수',
       '남30,40대생활인구수', '여30,40대생활인구수', '남50,60대생활인구수', '여50,60대생활인구수',
       '남자70세이상생활인구수', '여자70세이상생활인구수', '세대', '인구', '세대당인구',
       '외국인인구비율', '구내전입', '구내전출', '구간전입', '구간전출', '인구밀도', 'park', 'tree',
       'river']]
data = data.rename(columns={'미세먼지(㎍/㎥)':"finedust","기온(℃)":"temp","상대습도( %)":"humid","버스승차승객수": "busin", "버스하차승객수": "busout","대여":"bikerent","반납":"bikereturn",'남10대생활인구수':"m10", '여10대생활인구수':"f10", '남20대생활인구수':"m20", '여20대생활인구수':"f20",'남30,40대생활인구수':"m3040", '여30,40대생활인구수':"f3040", '남50,60대생활인구수':"m5060", '여50,60대생활인구수':"f5060",
       '남자70세이상생활인구수':"m70", '여자70세이상생활인구수':"f70", '세대':"fam", '인구':"pop", '세대당인구':"famperpop",
       '외국인인구비율':"foreign_rt", '구내전입':"gu_in", '구내전출':"gu_out", '구간전입':"gugan_in", '구간전출':"gugan_out", '인구밀도':"pop_dens"})
data = data.dropna()
data["bus"] = (data["busin"]+data["busout"])/2
data["bike"] = (data["bikerent"]+data["bikereturn"])/2
data["trans"] = (data["subway"]+data["parking"])
data["p10"] = data["m10"]+ data["f10"]
data["p20"] = data["m20"]+ data["f20"]
data["p3040"] = data["m3040"]+ data["f3040"]
data["p5060"] = data["m5060"]+ data["f5060"]
data["p70"] = data["m70"]+ data["f70"]
data = data.drop(columns = ['busin', 'busout', 'bikerent', 'bikereturn',"subway","parking",'m10', 'f10', 'm20', 'f20',
       'm3040', 'f3040', 'm5060', 'f5060', 'm70', 'f70'])
data["p20_60"] = data["p20"]+ data["p3040"]+ data["p5060"]
data =data.drop(columns = ["p20","p3040","p5060","fam","gugan_in"])


In [32]:
data = pd.read_csv("/Users/seungji/Desktop/dataanalysis/project/data/data_spatial.csv")

In [33]:
data.head()

,finedust,lat,lon,temp,humid,traffic,road,vent,pop,famperpop,...,pop_dens,park,tree,river,bus,bike,trans,p10,p70,p20_60
0,82.208333,37.494403,126.831472,15.192361,45.857639,622.579456,1528,26,24141.0,2.39,...,9100.0,2,549,3,40.446350,86.750000,249,2741.313970,2569.435972,16091.832318
1,47.540268,37.509299,126.878380,16.516779,42.315436,1112.406389,6617,56,36132.0,2.68,...,24826.0,1,347,6,51.694521,411.250000,377,2828.445961,2535.264541,24161.276830
2,42.948052,37.496892,126.843727,14.998052,46.321429,622.579456,2559,33,23434.0,1.90,...,39593.0,3,373,0,101.409529,366.666667,291,1171.538099,1973.291742,11218.724433
3,46.343972,37.487430,126.838527,14.177305,48.251773,622.579456,1923,27,39209.0,2.48,...,12423.0,2,320,0,124.349281,243.500000,257,4890.250857,6082.292009,34605.189729
4,77.467811,37.482145,126.889339,15.512446,44.381974,1063.823961,6954,40,13447.0,1.54,...,35658.0,0,666,0,91.170588,225.500000,319,277.104519,1035.259155,9625.592185


In [34]:
y = data["finedust"]
X = data.iloc[:,1:] 

from sklearn.model_selection import train_test_split
X_tr, X_tst, y_tr, y_tst = train_test_split(X,y,test_size = 0.2,random_state = 42)
print(X_tr.shape,y_tr.shape,X_tst.shape,y_tst.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_tr)

X_tr = scaler.transform(X_tr)
X_tst = scaler.transform(X_tst)

X_tr = pd.DataFrame(X_tr)
X_tst = pd.DataFrame(X_tst)

X_tr.columns = data.columns[1:]
X_tst.columns = data.columns[1:]

(844, 23) (844,) (212, 23) (212,)


### Ridge Lasso Regression

### Lasso

In [35]:
from sklearn import linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [36]:
lasso = LassoCV(cv = 5,random_state = 42)
lasso = lasso.fit(X_tr,y_tr)

In [37]:
y_tr_pred = lasso.predict(X_tr)
y_tst_pred = lasso.predict(X_tst)
lasso_tr_mse = mean_squared_error(list(y_tr),list(y_tr_pred))
lasso_tst_mse = mean_squared_error(list(y_tst),list(y_tst_pred))
print("lasso_tr_mse:",lasso_tr_mse)
print("lasso_tst_mse:",lasso_tst_mse)

lasso_tr_mse: 218.30492539808301
lasso_tst_mse: 179.54151426878423


In [38]:
pd.DataFrame({"col":X_tr.columns,"coef":lasso.coef_}).sort_values("coef")

,col,coef
2,temp,-3.279149
0,lat,-1.728408
3,humid,-1.344969
11,gu_out,-0.585660
5,road,-0.566040
12,gugan_out,-0.273764
1,lon,-0.130151
4,traffic,0.000000
20,p10,-0.000000
17,bus,0.000000


In [ ]:
lasso.

In [39]:
r2_score(list(y_tst),y_tst_pred)

0.06626066894948024

In [40]:
r2_score(list(y_tr),y_tr_pred)

0.08389889709115006

In [41]:
pd.concat([pd.DataFrame(list(y_tst)),pd.DataFrame(y_tst_pred)],axis=1)

,0,0
0,51.217391,51.694419
1,78.835664,49.264692
2,37.194719,52.576088
3,58.404762,56.954235
4,53.244300,53.486658
...,...,...
207,52.650155,51.178810
208,64.758865,56.941970
209,54.105442,51.819464
210,34.346505,52.420699


### Ridge

In [42]:
ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
ridge = ridge.fit(X_tr,y_tr)
y_tr_pred = ridge.predict(X_tr)
y_tst_pred = ridge.predict(X_tst)
ridge_tr_mse = mean_squared_error(y_tr,y_tr_pred)
ridge_tst_mse = mean_squared_error(y_tst,y_tst_pred)
print("ridge_tr_mse:",ridge_tr_mse)
print("ridge_tst_mse:",ridge_tst_mse)

ridge_tr_mse: 215.539266877795
ridge_tst_mse: 176.94155487895708


In [43]:
pd.DataFrame({"col":X_tr.columns,"ridge_coef":ridge.coef_,"lasso_coef":lasso.coef_}).sort_values("lasso_coef")

,col,ridge_coef,lasso_coef
2,temp,-4.024165,-3.279149
0,lat,-1.374797,-1.728408
3,humid,-2.060925,-1.344969
11,gu_out,-1.746001,-0.585660
5,road,-1.503761,-0.566040
12,gugan_out,-1.334512,-0.273764
1,lon,-1.608753,-0.130151
4,traffic,0.269676,0.000000
20,p10,0.206838,-0.000000
17,bus,0.048568,0.000000


In [44]:
r2_score(list(y_tst),y_tst_pred)

0.07978224556814095

In [46]:
r2_score(list(y_tr),y_tr_pred)

0.09550478649600447

In [45]:
pd.concat([pd.DataFrame(list(y_tst)),pd.DataFrame(y_tst_pred)],axis=1)

,0,0
0,51.217391,51.626107
1,78.835664,50.379793
2,37.194719,54.305223
3,58.404762,55.302850
4,53.244300,53.770508
...,...,...
207,52.650155,52.158495
208,64.758865,58.223760
209,54.105442,51.556230
210,34.346505,50.402318


In [48]:
X_tr.shape

(844, 23)

In [50]:
X_tr.head()

,lat,lon,temp,humid,traffic,road,vent,pop,famperpop,foreign_rt,...,pop_dens,park,tree,river,bus,bike,trans,p10,p70,p20_60
0,0.762238,-0.026597,-1.935445,3.589369,-0.857919,0.277849,-0.155213,-2.157262,-0.420939,1.414123,...,-1.738837,1.060531,-1.822852,-0.372401,-0.586821,-0.822867,0.793821,-1.207730,-1.964655,-1.425126
1,-0.569365,-0.104498,0.267560,0.053511,1.163996,-0.572596,0.593701,0.504298,1.557224,-0.067620,...,-1.044667,-0.525099,-0.615963,2.992167,-0.084268,-0.378174,1.662046,0.180605,0.576043,-0.011618
2,1.182982,0.207194,-0.077031,0.300816,-0.378012,1.428900,-0.364910,-0.755837,-0.610626,0.851029,...,-1.518968,-1.317914,-0.835397,0.749122,-0.379065,-0.943804,0.509535,-0.359263,-0.719294,-0.793532
3,2.399250,0.672539,0.485515,-0.181494,-0.439045,-1.182487,-1.503260,0.401822,0.337809,-0.547257,...,0.952154,-1.317914,0.745822,-0.372401,-0.576122,-0.335198,-1.726335,-0.538899,0.502032,-0.540838
4,-1.344701,-0.165835,-0.111306,0.015371,1.454475,-0.899583,-0.394866,-0.952610,-0.420939,-0.406220,...,1.358006,0.267716,-0.899937,-0.652782,-1.313116,0.375949,-0.189654,-0.949181,-1.109700,-0.985831


In [49]:
X_tst.shape

(212, 23)

In [56]:
m = folium.Map(
    location=[37.54238169591348,126.99068745821812],
 attr = '서울',
    zoom_start=12
)

for i in range(len(X_tr)):
    folium.Circle(
    location = [X_tr.lat[i],X_tr.lon[i]],
    radius = 3,
    color = "black").add_to(m)
folium.plugins.Fullscreen(position='topright',  
                   title='FULL SCREEN ON', 
                   title_cancel='FULL SCREEN OFF',
                   force_separate_button=True
                  ).add_to(m)
m